In [1]:
print("OKAY")

OKAY


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

c:\Users\furkanbaba\anaconda3\envs\spartacus\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = "gcp-starter"

In [5]:
def data_loader(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [6]:
extracted_data= data_loader("data/")

In [7]:
#extracted_data

In [8]:
def text_splitter(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
    text_chunks = splitter.split_documents(extracted_data)
    return text_chunks


In [9]:
text_chunks = text_splitter(extracted_data)
print("length of text:", len(text_chunks))

length of text: 3367


In [10]:
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_embeddings()

In [12]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [13]:
query_results = embeddings.embed_query("HI, how are you feeling today ?")
print("length:",len(query_results))

length: 384


In [14]:
query_results

[-0.04761326313018799,
 0.012561398558318615,
 0.08829054236412048,
 0.016965650022029877,
 0.04545752331614494,
 -0.04732634872198105,
 0.09706729650497437,
 -0.01631682924926281,
 -0.05710430443286896,
 -0.007524485699832439,
 -0.04716635122895241,
 0.011600297875702381,
 -0.017519285902380943,
 0.019553260877728462,
 0.03404289856553078,
 -0.003682906273752451,
 0.004642177373170853,
 -0.09220511466264725,
 -0.12892168760299683,
 0.08274782449007034,
 -0.08603700250387192,
 -0.013935116119682789,
 -0.05553831160068512,
 0.09202856570482254,
 0.01082191988825798,
 0.052058979868888855,
 0.0026824248488992453,
 0.009897642768919468,
 -0.011292709037661552,
 -0.11285684257745743,
 -0.04507773369550705,
 0.026827633380889893,
 -0.022926315665245056,
 -0.022284774109721184,
 -0.004771487321704626,
 0.025244969874620438,
 -0.0174946840852499,
 -0.16253238916397095,
 -0.004957687109708786,
 -0.011241892352700233,
 0.03618461638689041,
 -0.024022530764341354,
 0.017015555873513222,
 -0.0037

In [15]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index('medical-chatbot') 

In [16]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.12101,
 'namespaces': {'': {'vector_count': 12101}},
 'total_vector_count': 12101}

In [17]:
index_name="medical-chatbot"
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [58]:
query = "What is  acne?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Resources
BOOKS
A Manual of Laboratory and Diagnostic Tests. 5th ed. Ed.
Francis Fishback. Philadelphia: Lippincott, 1996.
Garza, Diana, and Kathleen Becan-McBride. Phlebotomy
Handbook. 4th ed. Stamford: Appleton & Lange, 1996.
PERIODICALS
Moul, Judd W., et al. “The Contemporary Value of Pretreat-
ment Prostatic Acid Phosphatase to Predict PathologicalStage and Recurrence in Radical Prostatectomy Cases.”Journal of Urology (Mar. 1998): 935-940.
Nancy J. Nordenson
Acid reflux seeHeartburn
Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
the most common skin disease. It affects nearly 17 millionpeople in the United States. While acne can arise at anyage, it usually begins at puberty and worsens during ado-


In [59]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [60]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [61]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0 (1).bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [62]:
retriever=docsearch.as_retriever(search_kwargs={'k': 2})

In [42]:
combine_docs_chain = create_stuff_documents_chain(llm, PROMPT)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [63]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])